In [1]:
# setting my si data frame
import pandas as pd
data = '/workspaces/service-data/outputs/snapshots/2025-03-01/si.csv'
#si = pd.read_csv(data, sep=';')
si = pd.read_csv(data,
                  keep_default_na=False, 
                  na_values='', 
                  delimiter=';',
                 engine='python',
                skipfooter=2
                )
si_dp = si

from datetime import datetime
import pytz
import numpy as np

# Specify the timezone
timezone = pytz.timezone("America/Montreal")

# Get the current date and time in the specified timezone
current_date = datetime.now(timezone)

# Format the current date and time into the desired string format
current_datestr = current_date.strftime("%Y-%m-%d_%H:%M:%S")

# Print the current date and time
print(f"current date: {current_datestr}")

#si_dp = si #ge: Use si_dp in your calculations to keep a clean si for other stuff

# si = pd.read_csv("https://github.com/gcperformance/service-data/releases/latest/download/si.csv", 
#                  keep_default_na=False, 
#                  na_values='', 
#                  delimiter=';',
#                  engine='python',
#                  skipfooter=2
#                  )


current date: 2025-05-01_13:49:37


In [2]:
# Create 'phone_apps_enquiries' column (sum of phone enquiries + phone applications)
# Convert relevant columns to numeric and handle NaNs in the calculation
si_dp['num_phone_enquiries'] = pd.to_numeric(si_dp['num_phone_enquiries'], errors='coerce').fillna(0)
si_dp['num_applications_online'] = pd.to_numeric(si_dp['num_applications_online'], errors='coerce').fillna(0)
si_dp['num_applications_in_person'] = pd.to_numeric(si_dp['num_applications_in_person'], errors='coerce').fillna(0)

si_dp['phone_apps_enquiries'] = (
    pd.to_numeric(si_dp['num_phone_enquiries'], errors='coerce').fillna(0) + 
    pd.to_numeric(si_dp['num_applications_by_phone'], errors='coerce').fillna(0)
)

# Create 'total_transactions' by summing all relevant application methods
si_dp['total_transactions'] = (
    pd.to_numeric(si_dp['num_applications_total'], errors='coerce').fillna(0) +
    pd.to_numeric(si_dp['num_phone_enquiries'], errors='coerce').fillna(0)
)

# adding a new column for applications done by phone, online and in person only
si_dp['apps_online_and_per'] = (
    pd.to_numeric(si['num_applications_in_person'], errors='coerce').fillna(0) + 
    pd.to_numeric(si['num_applications_online'], errors='coerce').fillna(0) + 
    pd.to_numeric(si['phone_apps_enquiries'], errors='coerce').fillna(0)
)


si_dp['omnichannel'] = si_dp.apply(
     lambda row: (row['phone_apps_enquiries'] >=0 and row['num_applications_online'] >=0 and row['num_applications_in_person'] >=0),
     axis=1
 )

# adding a new column for external
si_dp['external'] = si_dp['service_scope'].str.contains('EXTERN', na=False)

# adding a new column for high volume services
si_dp['highvolume'] = si_dp['total_transactions'] >= 45000

# adding a new column for online enabled Y
# creating columns to check which lists out the columns from os_account_registration to os_issue_resolution_feedback
columns_to_check = [ 'os_account_registration', 'os_authentication', 'os_application', 'os_decision', 'os_issuance', 'os_issue_resolution_feedback']
si_dp['online_enabledY'] = si_dp[columns_to_check].apply(lambda row: (row == 'Y').sum(), axis=1)

# adding column for online enabled N
si_dp['online_enabledN'] = si_dp[columns_to_check].apply(lambda row: (row == 'N').sum(), axis=1)

# adding column for online enabled NA
si_dp['online_enabledNA'] = si_dp[columns_to_check].isna().sum(axis=1)

# adding a new column for online end to end
si_dp['onlineE2E'] = si_dp.apply(
    lambda row: (row['online_enabledNA'] + row['online_enabledY'] == 6) and (row['online_enabledY'] > 0),
    axis=1
)

# adding a new column for online one or more points
si_dp['oip'] = si_dp['online_enabledY'].apply(lambda x: True if x > 0 else False)

# Importing the service standards data
ss_data = '/workspaces/service-data/outputs/snapshots/2025-03-01/ss.csv' 
ss = pd.read_csv(ss_data, sep=';')
ss_dp = ss


In [ ]:
# to add a new column for phone apps inquiries
# phone apps inquiries are num_phone_enquiries plus num_applications_by_phone
# based on error received 'can only concatenate str (not int) to string

# Columns to convert to numeric and fill NaNs with 0 (without doing it upfront)
# cols_to_numeric = [
#     'num_phone_enquiries',
#     'num_applications_by_phone',
#     'num_applications_by_email',
#     'num_applications_by_fax',
#     'num_applications_by_mail',
#     'num_applications_by_other',
#     'num_applications_in_person',
#     'num_applications_online',
#     'num_applications_total'
# ]

# Create 'phone_apps_inquiries' column (sum of phone enquiries + phone applications)
# Convert relevant columns to numeric and handle NaNs in the calculation
si_dp['phone_apps_inquiries'] = (
    pd.to_numeric(si['num_phone_enquiries'], errors='coerce').fillna(0) + 
    pd.to_numeric(si['num_applications_by_phone'], errors='coerce').fillna(0)
)

# Create 'total_transactions' by summing all relevant application methods
si_dp['total_transactions'] = (
    pd.to_numeric(si['num_applications_total'], errors='coerce').fillna(0) +
    pd.to_numeric(si['num_phone_enquiries'], errors='coerce').fillna(0)
)

# adding a new column for applications done by phone, online and in person only
si_dp['apps_online_and_per'] = (
    pd.to_numeric(si['num_applications_in_person'], errors='coerce').fillna(0) + 
    pd.to_numeric(si['num_applications_online'], errors='coerce').fillna(0) + 
    pd.to_numeric(si['phone_apps_inquiries'], errors='coerce').fillna(0)
)

# adding a new column for omnichannels
si_dp['omnichannel'] = si.apply(
    lambda row: 1 if pd.notna(row['phone_apps_inquiries']) and pd.notna(row['num_applications_online']) and pd.notna(row['num_applications_in_person']) else 0, 
    axis=1
)

# GE: test this way # adding a new column for omnichannels
# si['omnichannel'] = si.apply(
#     lambda row: (row['phone_apps_inquiries']>=0 and row['num_applications_online']>=0 and row['num_applications_in_person']>=0),
#     axis=1
# )

# adding a new column for external
si['external'] = si['service_scope'].str.contains('EXTERN', na=False).astype(int) #GE: keep true/false as boolean, no need to convert to int
#convert the column to numeric 
si['external'] = pd.to_numeric(si['external'], errors='coerce') #GE: possibly unnecessary

# adding a new column for high volume services
si['highvolume'] = (si['total_transactions'] >= 45000).astype(int) #GE: keep true/false as boolean, no need to convert to int

# adding a new column for online enabled Y
# creating columns to check which lists out the columns from os_account_registration to os_issue_resolution_feedback
columns_to_check = [ 'os_account_registration', 'os_authentication', 'os_application', 'os_decision', 'os_issuance', 'os_issue_resolution_feedback']
si['online_enabledY'] = si[columns_to_check].apply(lambda row: (row == 'Y').sum(), axis=1)

# adding column for online enabled N
si['online_enabledN'] = si[columns_to_check].apply(lambda row: (row == 'N').sum(), axis=1)

# adding column for online enabled NA
si['online_enabledNA'] = si[columns_to_check].isna().sum(axis=1)

# adding a new column for online end to end #GE: keep this as boolean, don't convert to int
si['onlineE2E'] = (
    si.apply(lambda row: "0" if row['online_enabledNA'] == 6 
    else "1" if row['online_enabledY'] + row['online_enabledNA'] == 6 
    else "0", axis=1)
    )
#GE: check against process.py line 255 (maf5) 
# adding a new column for online one or more points
si['onl_morepoints'] = si['online_enabledY'].apply(lambda x: '1' if x >= 1 else '0') #GE: convert to boolean, don't bother sending to int

# Importing the service standards data
ss_data = '/workspaces/service-data/outputs/ss.csv' 
ss = pd.read_csv(ss_data, sep=';') #GE: suggest loading to ss, then doing calcs on ss_dp, or something like that

# Adding column for services with standards and standards met #GE: Compare to the process.py script, line 225 (Maf2)
# Grouping and summarizing the 'ss' DataFrame
ss_count = ss.groupby(['service_id', 'fiscal_yr']).agg(
    standards_count=('service_id', 'size'),  # Count the occurrences #GE: double check this is the right aggregator function (size)
    standards_met=('target_met', lambda x: (x == 'Y').sum())  # Count where target_met is 'Y'
).reset_index()

# Merging the 'ss_count' DataFrame with the 'si' DataFrame
si = si.merge(ss_count[['service_id', 'fiscal_yr', 'standards_count', 'standards_met']], #GE: use fy_org_id_service_id field as keys
               on=['service_id', 'fiscal_yr'], 
               how='left')

# Replacing NaN values in 'standards_count' and 'standards_met' with 0
si['standards_count'] = si['standards_count'].fillna(0)
si['standards_met'] = si['standards_met'].fillna(0)

# Adding a new column for services that met at least one standard
si['STDS_metsome'] = (si['standards_met'] >= 1).astype(int) #GE: boolean

# Creating the FYSID column by merging fiscal_yr and service_id columns
si['FYSID'] = si['fiscal_yr'].astype(str) + si['service_id'].astype(str) #GE: use existing fy_org_id_service_id field

# --- MODIFIED SECTION BELOW ---

# Merge the 'ss' DataFrame with 'external' and 'highvolume' from 'si' using 'fy_org_id_service_id'
merged_df = pd.merge(
    ss, 
    si[['fy_org_id_service_id', 'external', 'highvolume']], 
    on='fy_org_id_service_id', 
    how='left'
)

# Create the 'external_service' column based on the merged data
merged_df['external_service'] = (merged_df['external'] == 1).astype(int)

# Add both 'external_service' and 'highvolume' columns back into the original 'ss' DataFrame
ss['external_service'] = merged_df['external_service']
ss['highvolume'] = merged_df['highvolume']


In [ ]:
# METRICs 2-16 - the transactions table will contain all the metrics that apply to external services alone

# Filter the si DataFrame where 'external' == 1
filtered_si = si_dp[si_dp['external'] == 1].copy() #GE: good idea to use copy, remember to use si_dp

# Ensure relevant columns are numeric
cols_to_numeric = [
    'total_transactions', 
    'num_applications_online', 
    'phone_apps_inquiries', 
    'num_applications_by_mail',
    'num_applications_in_person'
]
filtered_si[cols_to_numeric] = filtered_si[cols_to_numeric].apply(pd.to_numeric, errors='coerce').fillna(0)

# Group by fiscal_yr and sum the numeric columns
grouped = filtered_si.groupby('fiscal_yr')[cols_to_numeric].sum().reset_index()

# Rename columns #ge: dont bother
Transactions_table = grouped.rename(columns={
    'fiscal_yr': 'fiscal_year',
    'num_applications_online': 'online applications',
    'phone_apps_inquiries': 'phone applications',
    'num_applications_by_mail': 'mail applications',
    'num_applications_in_person': 'in_person_apps'
})

# Add share columns
Transactions_table['online_share'] = (
    Transactions_table['online applications'] / Transactions_table['total_transactions']
).fillna(0)

Transactions_table['phone_share'] = (
    Transactions_table['phone applications'] / Transactions_table['total_transactions']
).fillna(0)

Transactions_table['in_person_share'] = (
    Transactions_table['in_person_apps'] / Transactions_table['total_transactions']
).fillna(0)

# METRIC 4-5C OMNICHANNEL OFFERINGS

# Ensure relevant columns in filtered_si are numeric
cols_omni = ['total_transactions', 'num_applications_online', 'phone_apps_inquiries', 'num_applications_in_person']
filtered_si[cols_omni] = filtered_si[cols_omni].apply(pd.to_numeric, errors='coerce').fillna(0)

# Count distinct omnichannel services per fiscal year
omni_count_by_year = si[si['omnichannel'] == 1].groupby('fiscal_yr')['service_id'].nunique()

# Count total distinct services per fiscal year
total_count_by_year = si.groupby('fiscal_yr')['service_id'].nunique()

# Calculate omnichannel service share
share_omni_by_year = (omni_count_by_year / total_count_by_year) * 100

# === Metric 5a: Online as a share of omnichannel usage ===
total_transactions_by_year_omni = filtered_si[filtered_si['omnichannel'] == 1].groupby('fiscal_yr')['total_transactions'].sum()
sum_online_apps_by_year = filtered_si[filtered_si['omnichannel'] == 1].groupby('fiscal_yr')['num_applications_online'].sum()
share_online_by_year = (sum_online_apps_by_year / total_transactions_by_year_omni) * 100

# === Metric 5b: Phone as a share of omnichannel usage ===
sum_phone_by_year = filtered_si[filtered_si['omnichannel'] == 1].groupby('fiscal_yr')['phone_apps_inquiries'].sum()
share_phone_by_year = (sum_phone_by_year / total_transactions_by_year_omni) * 100

# === Metric 5c: In-person as a share of omnichannel usage ===
sum_in_person_by_year = filtered_si[filtered_si['omnichannel'] == 1].groupby('fiscal_yr')['num_applications_in_person'].sum()
share_in_person_by_year = (sum_in_person_by_year / total_transactions_by_year_omni) * 100

# === Reindex for consistent fiscal years ===
all_fiscal_years = Transactions_table['fiscal_year']

Transactions_table['Omnichannel_Service_Count'] = omni_count_by_year.reindex(all_fiscal_years).values
Transactions_table['Total_Service_Count'] = total_count_by_year.reindex(all_fiscal_years).values
Transactions_table['GC_services_with_omnichannel (%)'] = share_omni_by_year.reindex(all_fiscal_years).values

Transactions_table['Total_Transactions_Omnichannel'] = total_transactions_by_year_omni.reindex(all_fiscal_years).values
Transactions_table['omnichannel_online_apps'] = sum_online_apps_by_year.reindex(all_fiscal_years).values
Transactions_table['Online_Share_of_Omnichannel_Usage (%)'] = share_online_by_year.reindex(all_fiscal_years).values

Transactions_table['Omnichannel_phone_apps'] = sum_phone_by_year.reindex(all_fiscal_years).values
Transactions_table['Phone_Share_of_Omnichannel_Usage (%)'] = share_phone_by_year.reindex(all_fiscal_years).values

Transactions_table['omnichannel_inperson_apps'] = sum_in_person_by_year.reindex(all_fiscal_years).values
Transactions_table['In-Person_Share_of_Omnichannel_Usage (%)'] = share_in_person_by_year.reindex(all_fiscal_years).values

# METRIC 6-9 NUMBER OF External departments, programs, services, and high volume services
# METRIC 6: Number of External Departments 
no_departments_by_year = filtered_si.groupby('fiscal_yr')['department_en'].nunique()
no_departments_by_year = no_departments_by_year.reindex(Transactions_table['fiscal_year']).values
Transactions_table['external_departments'] = no_departments_by_year

# METRIC 7: Number of Programs 
no_programs_by_year = programs.groupby('fiscal_yr')['program_id'].nunique()
no_programs_by_year = no_programs_by_year.reindex(Transactions_table['fiscal_year']).values
Transactions_table['number_of_programs'] = no_programs_by_year

# METRIC 8: Number of External Services 
no_external_service_by_year = filtered_si.groupby('fiscal_yr')['service_id'].nunique()
no_external_service_by_year = no_external_service_by_year.reindex(Transactions_table['fiscal_year']).values
Transactions_table['external_services'] = no_external_service_by_year

# METRIC 9: Number of High Volume Services 
no_highvolume_services_by_year = filtered_si[filtered_si['highvolume'] == 1].groupby('fiscal_yr')['service_id'].nunique()
no_highvolume_services_by_year = no_highvolume_services_by_year.reindex(Transactions_table['fiscal_year']).values
Transactions_table['high_volume_services'] = no_highvolume_services_by_year

# METRIC 10-13 - total online, phone, in-person and mail transactions in millions
# the columns with this information already exist from metric 2-3c.

# METRIC 14-15 SHARE of external services online end-to-end and share of external services with at least one point online

filtered_si['onlineE2E'] = pd.to_numeric(filtered_si['onlineE2E'], errors='coerce')
filtered_si['onl_morepoints'] = pd.to_numeric(filtered_si['onl_morepoints'], errors='coerce')

# Define flags (external services which are online end-to-end and external services which have at least one point online)
filtered_si['onlinee2e'] = np.where((filtered_si['onlineE2E'] == 1), 1, 0)
filtered_si['one_point_onl'] = np.where((filtered_si['onl_morepoints'] == 1), 1, 0)
filtered_si['all_external'] = np.where(filtered_si['external'] == 1, 1, 0)

# Group by fiscal year and calculate the counts
online_services = filtered_si.groupby('fiscal_yr')[['onlinee2e', 'one_point_onl', 'all_external']].sum()

# Ensure columns involved in percentage calculations are numeric
online_services['onlinee2e'] = pd.to_numeric(online_services['onlinee2e'], errors='coerce').fillna(0)
online_services['one_point_onl'] = pd.to_numeric(online_services['one_point_onl'], errors='coerce').fillna(0)
online_services['all_external'] = pd.to_numeric(online_services['all_external'], errors='coerce').fillna(0)

# Calculate percentages safely with numeric columns
online_services['pct_onlinee2e'] = (online_services['onlinee2e'] / online_services['all_external']) * 100
online_services['pct_one_point_onl'] = (online_services['one_point_onl'] / online_services['all_external']) * 100

# Align with Transactions_table fiscal years
online_services = online_services.reindex(Transactions_table['fiscal_year'])

# Add to Transactions_table
Transactions_table['service_onlineE2E_%'] = online_services['pct_onlinee2e'].values
Transactions_table['at_least_1point_onl_%'] = online_services['pct_one_point_onl'].values

# METRIC 16 external services standards meeting targets
# Create new column for standards meeting target
ss['standards_meeting_target'] = np.where((ss['target_met'] == 'Y') & (ss['external_service'] == 1), 1, 0)

# Create new column for total standards
ss['total_standards'] = np.where((ss['target_met'].notna()) & (ss['external_service'] == 1), 1, 0)

# Group by fiscal year and calculate the sum for each metric
standards_meeting_targets = ss.groupby('fiscal_yr')[['standards_meeting_target', 'total_standards']].sum()

# Calculate the percentage of standards meeting the target
standards_meeting_targets['pct_standards_meeting_target'] = (
    standards_meeting_targets['standards_meeting_target'] / standards_meeting_targets['total_standards']
) * 100

# Align with Transactions_table fiscal years
standards_meeting_targets = standards_meeting_targets.reindex(Transactions_table['fiscal_year'])

# Add to Transactions_table
Transactions_table['Standards_Meeting_Targets%'] = standards_meeting_targets['pct_standards_meeting_target'].values

# Final output
print(Transactions_table)

# Export the Transactions_table to a CSV file
Transactions_table.to_csv('Transactions_table.csv', index=True)

# Transpose the Transactions_table such that fiscal_year becomes the columns
Transactions_table_transposed = Transactions_table.set_index('fiscal_year').transpose()

# Final output: print the transposed table
print(Transactions_table_transposed)

# Export the transposed Transactions_table to a CSV file
Transactions_table_transposed.to_csv('Transactions_table_transposed.csv', index=True)








  fiscal_year  total_transactions  online applications  phone applications  \
0   2018-2019         421133885.0          174630378.0          53099291.0   
1   2019-2020         412398882.0          156831471.0          90704008.0   
2   2020-2021         386183263.0          202706431.0         137242208.0   
3   2021-2022         358629620.0          196235510.0         104413441.0   
4   2022-2023         464461678.0          268647250.0          72927098.0   
5   2023-2024         462846091.0          254970341.0          71119731.0   
6   2024-2025                 0.0                  0.0                 0.0   

   mail applications  in_person_apps  online_share  phone_share  \
0         42316481.0     118928362.0      0.414667     0.126086   
1         33221419.0     111052720.0      0.380291     0.219942   
2         19468112.0      15198693.0      0.524897     0.355381   
3         22377948.0      27955327.0      0.547182     0.291146   
4         31707932.0      78959836.0    

In [4]:
print(Transactions_table[['fiscal_year', 'Omnichannel_Service_Count', 'Total_Service_Count', 'GC_services_with_omnichannel (%)']])


  fiscal_year  Omnichannel_Service_Count  Total_Service_Count  \
0   2018-2019                       1242                 1244   
1   2019-2020                       1280                 1282   
2   2020-2021                       1426                 1426   
3   2021-2022                        161                 1610   
4   2022-2023                        132                 1690   
5   2023-2024                        967                 1688   
6   2024-2025                          1                    1   

   GC_services_with_omnichannel (%)  
0                         99.839228  
1                         99.843994  
2                        100.000000  
3                         10.000000  
4                          7.810651  
5                         57.286730  
6                        100.000000  


In [9]:
# METRIC 6-9 NUMBER OF External departments, programs, services, and high volume services

# Ensure grouping keys and any possibly numeric fields are treated correctly
filtered_si['fiscal_yr'] = pd.to_numeric(filtered_si['fiscal_yr'], errors='coerce')
programs['fiscal_yr'] = pd.to_numeric(programs['fiscal_yr'], errors='coerce')

# METRIC 6: Number of External Departments 
filtered_si['department_en'] = filtered_si['department_en'].astype(str)
no_departments_by_year = filtered_si.groupby('fiscal_yr')['department_en'].nunique()
no_departments_by_year = no_departments_by_year.reindex(Transactions_table['fiscal_year']).fillna(0).astype(int).values
Transactions_table['external_departments'] = no_departments_by_year

# METRIC 7: Number of Programs 
programs['program_id'] = programs['program_id'].astype(str)
no_programs_by_year = programs.groupby('fiscal_yr')['program_id'].nunique()
no_programs_by_year = no_programs_by_year.reindex(Transactions_table['fiscal_year']).fillna(0).astype(int).values
Transactions_table['number_of_programs'] = no_programs_by_year

# METRIC 8: Number of External Services 
filtered_si['service_id'] = filtered_si['service_id'].astype(str)
no_external_service_by_year = filtered_si.groupby('fiscal_yr')['service_id'].nunique()
no_external_service_by_year = no_external_service_by_year.reindex(Transactions_table['fiscal_year']).fillna(0).astype(int).values
Transactions_table['external_services'] = no_external_service_by_year

# METRIC 9: Number of High Volume Services 
filtered_si_highvolume = filtered_si[filtered_si['highvolume'] == 1]
filtered_si_highvolume['service_id'] = filtered_si_highvolume['service_id'].astype(str)
no_highvolume_services_by_year = filtered_si_highvolume.groupby('fiscal_yr')['service_id'].nunique()
no_highvolume_services_by_year = no_highvolume_services_by_year.reindex(Transactions_table['fiscal_year']).fillna(0).astype(int).values
Transactions_table['high_volume_services'] = no_highvolume_services_by_year

# METRIC 10-13 - total online, phone, in-person and mail transactions in millions
# Already handled in metrics 2–3c

# METRIC 14-15: Share of external services online end-to-end and with at least one point online

# Ensure 'fiscal_yr' is numeric
filtered_si['fiscal_yr'] = pd.to_numeric(filtered_si['fiscal_yr'], errors='coerce')

# Convert key columns to numeric safely
filtered_si['onlineE2E'] = pd.to_numeric(filtered_si['onlineE2E'], errors='coerce').fillna(0)
filtered_si['onl_morepoints'] = pd.to_numeric(filtered_si['onl_morepoints'], errors='coerce').fillna(0)

# Define flags for online capabilities
filtered_si['onlinee2e'] = np.where(filtered_si['onlineE2E'] == 1, 1, 0)
filtered_si['one_point_onl'] = np.where(filtered_si['onl_morepoints'] == 1, 1, 0)
filtered_si['all_external'] = 1  # all rows in filtered_si are already external

# Group by fiscal year and sum the flags
online_services = filtered_si.groupby('fiscal_yr')[['onlinee2e', 'one_point_onl', 'all_external']].sum().reset_index()

# Ensure numeric types for safe division
online_services['onlinee2e'] = pd.to_numeric(online_services['onlinee2e'], errors='coerce').fillna(0)
online_services['one_point_onl'] = pd.to_numeric(online_services['one_point_onl'], errors='coerce').fillna(0)
online_services['all_external'] = pd.to_numeric(online_services['all_external'], errors='coerce').replace(0, np.nan)  # avoid division by 0

# Calculate percentages
online_services['pct_onlinee2e'] = (online_services['onlinee2e'] / online_services['all_external']) * 100
online_services['pct_one_point_onl'] = (online_services['one_point_onl'] / online_services['all_external']) * 100

# Align with Transactions_table fiscal years
online_services = online_services.set_index('fiscal_yr').reindex(Transactions_table['fiscal_year']).fillna(0)

# Add results to Transactions_table
Transactions_table['service_onlineE2E_%'] = online_services['pct_onlinee2e'].values
Transactions_table['at_least_1point_onl_%'] = online_services['pct_one_point_onl'].values

# METRIC 16: External services standards meeting targets

# Ensure 'fiscal_yr' is numeric
ss['fiscal_yr'] = pd.to_numeric(ss['fiscal_yr'], errors='coerce')

# Create numeric flags for whether standards were met and whether the row counts toward totals
ss['standards_meeting_target'] = np.where((ss['target_met'] == 'Y') & (ss['external_service'] == 1), 1, 0)
ss['total_standards'] = np.where((ss['target_met'].notna()) & (ss['external_service'] == 1), 1, 0)

# Group by fiscal year and sum
standards_meeting_targets = ss.groupby('fiscal_yr')[['standards_meeting_target', 'total_standards']].sum().reset_index()

# Convert to numeric explicitly
standards_meeting_targets['standards_meeting_target'] = pd.to_numeric(standards_meeting_targets['standards_meeting_target'], errors='coerce').fillna(0)
standards_meeting_targets['total_standards'] = pd.to_numeric(standards_meeting_targets['total_standards'], errors='coerce').replace(0, np.nan)  # avoid division by 0

# Calculate percentage of standards met
standards_meeting_targets['pct_standards_meeting_target'] = (
    standards_meeting_targets['standards_meeting_target'] / standards_meeting_targets['total_standards']
) * 100

# Align with Transactions_table fiscal years
standards_meeting_targets = standards_meeting_targets.set_index('fiscal_yr').reindex(Transactions_table['fiscal_year']).fillna(0)

# Add results to Transactions_table
Transactions_table['Standards_Meeting_Targets%'] = standards_meeting_targets['pct_standards_meeting_target'].values

# Final output
# print(Transactions_table)

# Export the Transactions_table to a CSV file
Transactions_table.to_csv('Transactions_table.csv', index=True)

# Transpose the Transactions_table such that fiscal_year becomes the columns
Transactions_table_transposed = Transactions_table.set_index('fiscal_year').transpose()

# Final output: print the transposed table
# print(Transactions_table_transposed)

# Export the transposed Transactions_table to a CSV file
Transactions_table_transposed.to_csv('Transactions_table_transposed.csv', index=True)

Transactions_table_transposed

/tmp/ipykernel_28431/3798384946.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_si_highvolume['service_id'] = filtered_si_highvolume['service_id'].astype(str)


fiscal_year,2018-2019,2019-2020,2020-2021,2021-2022,2022-2023,2023-2024,2024-2025
total_transactions,4.211339e+08,4.123989e+08,3.861833e+08,3.586296e+08,4.644617e+08,4.628461e+08,0.0
online applications,1.746304e+08,1.568315e+08,2.027064e+08,1.962355e+08,2.686472e+08,2.549703e+08,0.0
phone applications,5.309929e+07,9.070401e+07,1.372422e+08,1.044134e+08,7.292710e+07,7.111973e+07,0.0
mail applications,4.231648e+07,3.322142e+07,1.946811e+07,2.237795e+07,3.170793e+07,2.140525e+07,0.0
in_person_apps,1.189284e+08,1.110527e+08,1.519869e+07,2.795533e+07,7.895984e+07,1.025862e+08,0.0
online_share,4.146671e-01,3.802907e-01,5.248970e-01,5.471815e-01,5.784056e-01,5.508750e-01,0.0
phone_share,1.260865e-01,2.199424e-01,3.553810e-01,2.911456e-01,1.570142e-01,1.536574e-01,0.0
in_person_share,2.824004e-01,2.692847e-01,3.935617e-02,7.795041e-02,1.700029e-01,2.216421e-01,0.0
Omnichannel_Service_Count,1.242000e+03,1.280000e+03,1.426000e+03,1.610000e+02,1.320000e+02,9.670000e+02,1.0
Total_Service_Count,1.244000e+03,1.282000e+03,1.426000e+03,1.610000e+03,1.690000e+03,1.688000e+03,1.0
